<b>
<center>
<h1>Predicting Readmission and Treatments for Diabetic Patients</h1> <br>
<h2>Final Project - Machine Learning I - Spring 2020 </h2><br>
<h3>Team members: Trinh Vu, Madhuri Yadav, Sharmin Kantharia</h3>
<h3>Part 2 - Predicting Treatments</h3>
</center>
</b>





# Introduction
We will work on Kaggle dataset [Diabetes 130 US hospitals for years 1999 - 2008](https://www.kaggle.com/brandao/diabetes) 

The data represents 10 years of clinical care at 130 US hospitals. It includes 50 features representing patient and hospital outcomes. Information was extracted from the database for encounters that satisfied the following criteria.

(1) It is an inpatient encounter (a hospital admission).

(2) It is a diabetic encounter, that is, one during which any kind of diabetes was entered to the system as a diagnosis.

(3) The length of stay was at least 1 day and at most 14 days.

(4) Laboratory tests were performed during the encounter.

(5) Medications were administered during the encounter.

The data contains such attributes as patient number, race, gender, age, admission type, time in hospital, medical specialty of admitting physician, number of lab test performed, HbA1c test result, diagnosis, number of medication, diabetic medications, number of outpatient, inpatient, and emergency visits in the year before the hospitalization, etc.

The target variables will be readmission and treatment. Our goals is to predict the readmission and treatment of a patient.

## Google Drive

In [0]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

# Set working directory
abspath = '/content/drive/My Drive/Colab Notebooks/ML1/Final Project/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Warning

In [0]:
import warnings
# Ignore warnings
warnings.filterwarnings('ignore')

# Data Preprocessing


## Loading data

In [0]:
import pandas as pd

# Load the raw data
df_raw = pd.read_csv(abspath + 'diabetic_data.csv', header = 0, na_values=['?'])

# Make a copy of df_raw
df = df_raw.copy(deep=True)

# Print the dimension of df
pd.DataFrame([[df.shape[0], df.shape[1]]], columns = ['# rows', '# columns'])

,# rows,# columns
0,101766,50


In [0]:
# Print the first 5 rows of df
df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),NaN,6,25,1,1,NaN,Pediatrics-Endocrinology,41,0,1,0,0,0,250.83,NaN,NaN,1,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),NaN,1,1,7,3,NaN,NaN,59,0,18,0,0,0,276,250.01,255,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),NaN,1,1,7,2,NaN,NaN,11,5,13,2,0,1,648,250,V27,6,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),NaN,1,1,7,2,NaN,NaN,44,1,16,0,0,0,8,250.43,403,7,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),NaN,1,1,7,1,NaN,NaN,51,0,8,0,0,0,197,157,250,5,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   encounter_id              101766 non-null  int64 
 1   patient_nbr               101766 non-null  int64 
 2   race                      99493 non-null   object
 3   gender                    101766 non-null  object
 4   age                       101766 non-null  object
 5   weight                    3197 non-null    object
 6   admission_type_id         101766 non-null  int64 
 7   discharge_disposition_id  101766 non-null  int64 
 8   admission_source_id       101766 non-null  int64 
 9   time_in_hospital          101766 non-null  int64 
 10  payer_code                61510 non-null   object
 11  medical_specialty         51817 non-null   object
 12  num_lab_procedures        101766 non-null  int64 
 13  num_procedures            101766 non-null  int64 
 14  num_

## Handling the Identifiers

### Identify the Identiers

In [0]:
# Define a function to get the identifiers
def id_checker(df):
  '''
  The function checks the identifiers
  Parameter df: dataframe
  Returns the dataframe of the identifers
  '''
  df_id = df[[var for var in df.columns 
              if df[var].nunique(dropna=True) == df[var].notnull().sum()]]
  return df_id

In [0]:
# Call the function on df
df_id = id_checker(df)

# Print the first 5 rows of df_id
df_id.head()

,encounter_id
0,2278392
1,149190
2,64410
3,500364
4,16680


## Separating 24 treatments into 2 categories 'insulin' and 'io' (stands for insulin + others) and create 1 column of Treatment.

### Use the identifier to separate treatments

In [0]:
# Create a dataframe contains only treatment and encounter_id
treatments = df[['encounter_id','metformin', 'repaglinide', 'nateglinide',
       'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
       'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose',
       'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton',
       'insulin', 'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone']].copy()

In [0]:
# Label values from treatment as 0 or 1. 0 means the medication is not prescribed.
# 1 means the medication is prescribed. 'Steady', 'Up', 'Down' are labled as 1
treatments=treatments.replace(['No','Steady','Up','Down'],[0,1,1,1])
treatments.set_index('encounter_id',inplace=True)

In [0]:
# Print first 5 rows of treatments
treatments.head()

,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone
encounter_id,,,,,,,,,,,,,,,,,,,,,,,
2278392,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
149190,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
64410,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
500364,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
16680,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [0]:
treatments.sum(axis=1).value_counts()

1    47314
0    23403
2    21873
3     7778
4     1335
5       58
6        5
dtype: int64

Patients are given combination of 6 medications (at maximum). 
We will encode the treatment so that when the patient is given only insulin, we label the treatment as 'insulin'. When patient is given insulin and other medications, we label it as 'io' (insulin + others). We label 'others' otherwise.

### Creating a new feature "Treatments"


#### 1. When the value of Insulin is '1' , create the classes "insulin" & "io" (insulin + others )

In [0]:
insulin1 = treatments[treatments['insulin']==1].sum(axis = 1).replace([1,2,3,4,5,6],['insulin','io','io','io','io','io'])

#### 2. When the value of Insuin is '0' , create the classes "others" & "no med"

In [0]:
insulin0 = treatments[treatments['insulin']==0].sum(axis=1).replace([0,1,2,3,4,5,6],['no med','other','other','other','other','other','other'])

In [0]:
# Combine insulin1 and insulin0
treatments=pd.concat([insulin1,insulin0])
treatments = pd.DataFrame({'treatments':treatments})

In [0]:
# Add treatments feature to the df dataframe
df=df.join(treatments,on='encounter_id') #setting index as encounter_id

In [0]:
# Now we have treatments as 1 column, we will remove the 24 treatments from df
df = df.drop(columns = ['metformin', 'repaglinide', 'nateglinide',
       'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
       'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose',
       'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton',
       'insulin', 'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone'])

### Remove the Identifier

In [0]:
# Remove the identifiers
df = df.drop(columns = 'encounter_id')

# Print the firt 5 rows of df
df.head()

,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,change,diabetesMed,readmitted,treatments
0,8222157,Caucasian,Female,[0-10),NaN,6,25,1,1,NaN,Pediatrics-Endocrinology,41,0,1,0,0,0,250.83,NaN,NaN,1,None,None,No,No,NO,no med
1,55629189,Caucasian,Female,[10-20),NaN,1,1,7,3,NaN,NaN,59,0,18,0,0,0,276,250.01,255,9,None,None,Ch,Yes,>30,insulin
2,86047875,AfricanAmerican,Female,[20-30),NaN,1,1,7,2,NaN,NaN,11,5,13,2,0,1,648,250,V27,6,None,None,No,Yes,NO,other
3,82442376,Caucasian,Male,[30-40),NaN,1,1,7,2,NaN,NaN,44,1,16,0,0,0,8,250.43,403,7,None,None,Ch,Yes,NO,insulin
4,42519267,Caucasian,Male,[40-50),NaN,1,1,7,1,NaN,NaN,51,0,8,0,0,0,197,157,250,5,None,None,Ch,Yes,NO,io


In [0]:
# Patient_nbr will also be removed because it doesn't effect the predictions of readmission or treatment.
df = df.drop(columns = 'patient_nbr')

## Handling missing data

### Identify the missing values

#### Find the variables with NaN, their proportion of NaN, and dtype

In [0]:
# Define a function to check variables with NaN
def nan_checker(df):
  '''
  The function checks for NaN
  Parameter df: dataframe
  Returns the dataframe of variables with NaN, their proportion of NaN, and dtype
  '''

  # Get the variables with NaN, their proportion of NaN and dtype
  df_nan = pd.DataFrame([[var, df[var].isna().sum() / df.shape[0], df[var].dtype]
                         for var in df.columns if df[var].isna().sum() > 0],
                        columns = ['var', 'proportion', 'dtype'])
  
  # Sort df_nan in descending order of the proportion of NaN
  df_nan = df_nan.sort_values(by = 'proportion', ascending = False)

  return df_nan

In [0]:
# Call the function nan_checker on df
df_nan = nan_checker(df)

# Print df_nan
df_nan.reset_index(drop=True)

,var,proportion,dtype
0,weight,0.968585,object
1,medical_specialty,0.490822,object
2,payer_code,0.395574,object
3,race,0.022336,object
4,diag_3,0.013983,object
5,diag_2,0.003518,object
6,diag_1,0.000206,object


There is no missing values that has numerical datatype. Variables 'weight', 'medical_specialty', and 'payer_code' have large proportions of NaN and they are object datatype, we will remove these three columns.

In [0]:
# Remove columns 'weight', 'medical_specialty', and 'payer_code'
df = df.drop(columns= ['weight', 'medical_specialty', 'payer_code'])

# Print the first 5 rows of df
df.head()

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,change,diabetesMed,readmitted,treatments
0,Caucasian,Female,[0-10),6,25,1,1,41,0,1,0,0,0,250.83,NaN,NaN,1,None,None,No,No,NO,no med
1,Caucasian,Female,[10-20),1,1,7,3,59,0,18,0,0,0,276,250.01,255,9,None,None,Ch,Yes,>30,insulin
2,AfricanAmerican,Female,[20-30),1,1,7,2,11,5,13,2,0,1,648,250,V27,6,None,None,No,Yes,NO,other
3,Caucasian,Male,[30-40),1,1,7,2,44,1,16,0,0,0,8,250.43,403,7,None,None,Ch,Yes,NO,insulin
4,Caucasian,Male,[40-50),1,1,7,1,51,0,8,0,0,0,197,157,250,5,None,None,Ch,Yes,NO,io


In [0]:
# Print the dimension of df
pd.DataFrame([[df.shape[0], df.shape[1]]], columns = ['# rows', '# columns'])

,# rows,# columns
0,101766,23


## Encoding the data

In [0]:
# Print the unique dtypes of variables in df
pd.DataFrame(df.dtypes.unique(), columns = ['dtype'])

,dtype
0,object
1,int64


### Identify Categorical Variables that have object as dtype


In [0]:
# Define a function to check for variables with object as dtype
def cat_var_checker(df):
  """
  The function checks for categorical variables
  Parameter df: dataframe

  Returns the dataframe of categorical variables and their number of unique value
  """

  # Get the dataframe of categorical variables and their number of unique value
  df_cat = pd.DataFrame([[var, df[var].nunique(dropna=False)] 
                        for var in df.columns if df[var].dtype == 'object'],
                        columns = ['var', 'nunique'])
  
  # Sort df_cat in descending order of the number of unique value
  df_cat = df_cat.sort_values(by='nunique', ascending=False).reset_index(drop=True)

  return df_cat

In [0]:
# Call the function on df
df_cat = cat_var_checker(df)

# Print df_cat
df_cat

,var,nunique
0,diag_3,790
1,diag_2,749
2,diag_1,717
3,age,10
4,race,6
5,max_glu_serum,4
6,A1Cresult,4
7,treatments,4
8,gender,3
9,readmitted,3


In [0]:
# Gender has 3 unique values, let's look at what they are.
df.gender.value_counts()

Female             54708
Male               47055
Unknown/Invalid        3
Name: gender, dtype: int64

In [0]:
# Remove unknown/invalid gender entries from df
df=df[df['gender']!='Unknown/Invalid']

### Remove categorical features with large number of categories in df

In [0]:
# Remove diag_3, diag_2, diag_1 because they have large number of uniques in df.
# Another reason is they don't contribute to the outcome of the predictions of either readmitted or treatments.
df = df.drop(columns = ['diag_1', 'diag_2', 'diag_3'])

### Imputing NaN with mode


In [0]:
# Imputing NaN with mode
df['race'].fillna(df['race'].mode()[0], inplace=True)

In [0]:
# Print the dimesion of df
pd.DataFrame([[df.shape[0], df.shape[1]]], columns = ['# rows', '# columns'])

,# rows,# columns
0,101763,20


In [0]:
# Print the first 5 rows of df
df.head()

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,max_glu_serum,A1Cresult,change,diabetesMed,readmitted,treatments
0,Caucasian,Female,[0-10),6,25,1,1,41,0,1,0,0,0,1,None,None,No,No,NO,no med
1,Caucasian,Female,[10-20),1,1,7,3,59,0,18,0,0,0,9,None,None,Ch,Yes,>30,insulin
2,AfricanAmerican,Female,[20-30),1,1,7,2,11,5,13,2,0,1,6,None,None,No,Yes,NO,other
3,Caucasian,Male,[30-40),1,1,7,2,44,1,16,0,0,0,7,None,None,Ch,Yes,NO,insulin
4,Caucasian,Male,[40-50),1,1,7,1,51,0,8,0,0,0,5,None,None,Ch,Yes,NO,io


# Predicting Treatments

## Getting the name of the target

In [0]:
target_treatments = 'treatments'

## Encoding the categorical features using pandas.get_dummies

In [0]:
import numpy as np
# One-hot-encode the categorical features in the combined data
df = pd.get_dummies(df, columns = np.setdiff1d(np.intersect1d(df.columns, df_cat['var']), [target_treatments]))

## Encoding the Categorical Target using sklearn.LabelEncoder

In [0]:
from sklearn.preprocessing import LabelEncoder

# The LabelEncoder
le = LabelEncoder()

# Encode the categorical target in the combined data
df[target_treatments] = le.fit_transform(df[target_treatments].astype(str))

## Dividing df into training and testing sets

In [0]:
from sklearn.model_selection import train_test_split
# Divide the training data into training (80%) and test (20%) using sklearn train_test_split with random_state=42 and stratify
df_train, df_test = train_test_split(df, train_size = 0.8, random_state = 42, stratify = df[target_treatments])

# Reset the index
df_train, df_test = df_train.reset_index(drop=True), df_test.reset_index(drop=True)

## Dividing df_train into training and validation sets

In [0]:
from sklearn.model_selection import train_test_split
# Divide the training data into training (80%) and test (20%) using sklearn train_test_split with random_state=42 and stratify
df_train, df_valid = train_test_split(df_train, train_size = 0.8, random_state = 42, stratify = df_train[target_treatments])

# Reset the index
df_train, df_valid = df_train.reset_index(drop=True), df_valid.reset_index(drop=True)

In [0]:
# Print the dimesion of df_train
pd.DataFrame([[df_train.shape[0], df_train.shape[1]]], columns = ['# rows', '# columns'])

,# rows,# columns
0,65128,45


In [0]:
# Print the dimension of df_valid
pd.DataFrame([[df_valid.shape[0], df_valid.shape[1]]], columns = ['# rows', '# columns'])

,# rows,# columns
0,16282,45


In [0]:
# Print the dimension of df_test
pd.DataFrame([[df_test.shape[0], df_test.shape[1]]], columns = ['# rows', '# columns'])

,# rows,# columns
0,20353,45


## Getting the name of the features

In [0]:
# Get the name of features
features = np.setdiff1d(df.columns, [target_treatments])

## Getting the feature matrix and target vector

In [0]:
# Get the feature matrix
X_train = df_train[features].to_numpy()
X_valid = df_valid[features].to_numpy()
X_test = df_test[features].to_numpy()

# Get the target vector
y_train = df_train[target_readmitted].astype(int).to_numpy()
y_valid = df_valid[target_readmitted].astype(int).to_numpy()

## Scaling the data

In [0]:
# Standardize the data using sklearn StandardScaler
from sklearn.preprocessing import StandardScaler

# The StandardScaler
ss = StandardScaler()

# Standardize the training data
X_train = ss.fit_transform(X_train)

# Standardize the validation data
X_valid = ss.transform(X_valid)

# Standardize the testing data
X_test = ss.transform(X_test)

## Hyperparameter Tuning and Model Selection